<h1><center> UFU - Federal University of Uberlândia</center></h1>

<h2><center>Undergraduate Program in Civil Engineering</center></h2>

<h3><center>SCIENTIFIC RESEARCH PROJECT</center><br>
TITLE: USING XGBOOST MODELS FOR DAILY RAINFALL PREDICTION 
<br>  
<br>  
STUDENT: Pedro Augusto Toledo Rios</h3>

<p>This notebook is part of a Scientific Research Project in the field of Computer Science/Data Analysis.</p>


# Notebook for Daily Rainfall Amount (mm) Determination - Global Dataset

## Imports and Initial Configurations

In [ ]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd
import seaborn as sn

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import  metrics
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
import itertools
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option("display.max_colwidth", 150)
pd.set_option("display.min_rows", 20)

import matplotlib.pyplot as plt

import seaborn as sns # Statistical data visualization

import plotly.graph_objects as go
import plotly.express as px
from keras.models import Sequential
from keras.layers import LSTM, SimpleRNN, Dense, Dropout, Masking, Embedding

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional

import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

## Exploratory Data Analysis

In [ ]:
# Tratamento de dados
missing_values = ['n/a','na','*****','*','*******',' -', '******', '5..84', '3..66', '3.3.21', '1..41', '********', '3.7.94', '354.59*', '564..79', '5.04.24', '21:36','**********', '***', '*********', '03:18', '00:00', '03:48', '08:42', '03:06', '09:06', '01:30', '07:48', '09:12', '10:18', '01:24', '#VALUE!' , '926,4923,8', '27/07/1902**21:36:00', '-', '926.4923.8']

dados_clima = pd.read_csv('C:/Users/auped/Desktop/IC CORREÇÕES/python 05-11/pesquisa/dadosclimaatt - CORRETO.csv', header = None, sep=';', na_values=missing_values)
dados_clima.columns=['Temp Máx (°C)', 'Temp Mín (°C)', 'Temp Média (°C)', 'Velocidade do Vento (km/h)', 'Radiação solar (cal/cm²/h)', 'Pressão (mb)', 'UR (%)', 'Chuva (mm)', 'Mês', 'Ano']
dados_clima['UR (%)'] = dados_clima['UR (%)'].str.replace(':', '.')
dados_clima['Pressão (mb)'] = dados_clima['Pressão (mb)'].str.replace(',,', '.')
dados_clima['UR (%)'] = dados_clima['UR (%)'].astype(float)
dados_clima['Pressão (mb)'] = dados_clima['Pressão (mb)'].astype(float)
dados_clima['Ano'] = dados_clima['Ano'].str.replace(',,', '')
dados_clima['Ano'] = dados_clima['Ano'].astype(int)

# Filtrar valores de pressão fora do intervalo válido
dados_clima = dados_clima[(dados_clima['Pressão (mb)'] >= 870) & (dados_clima['Pressão (mb)'] <= 1100)]

combine = [dados_clima]
chuva = dados_clima['Chuva (mm)']

# Primeiras cinco linhas dos dados pós-tratados
dados_clima.head()
dados_clima.dropna()

# Número de missing values encontrdos em cada coluna
print(dados_clima.isnull().sum())

In [ ]:
dados_clima.describe()

# Renomear as colunas para inglês
dados_clima_ingles = dados_clima.describe().rename(columns={
    'Temp Máx (°C)': 'Max Temperature (°C)',
    'Temp Mín (°C)': 'Min Temperature (°C)',
    'Temp Média (°C)': 'Average temperature (°C)',
    'Velocidade do Vento (km/h)': 'Wind Speed (km/h)',
    'Radiação solar (cal/cm²/h)': 'Solar Radiation (cal/cm²/h)',
    'Pressão (mb)': 'Pressure (mb)',
    'UR (%)': 'Relative humidity (%)',
    'Chuva (mm)': 'Daily rain (mm)'
})

# Salvar em Excel no caminho especificado
caminho = r"C:\Users\auped\Desktop\IC CORREÇÕES\dados_clima_describe_en.xlsx"
dados_clima_ingles.to_excel(caminho)



In [ ]:
dftabela = dados_clima.describe() 
dftabela.to_excel("tabelaestatisticaGLOBALL.xlsx")

### Implementation of a Rain Detection Class for Daily Weather Data

In [ ]:

# Loop que preenche a nova coluna com 0(não choveu) e 1(choveu)
for dados_clima in combine:    
    dados_clima.loc[ dados_clima['Chuva (mm)'] == 0, 'Choveu?'] = 0
    dados_clima.loc[(dados_clima['Chuva (mm)'] > 0), 'Choveu?'] = 1

#Visualização das primeiras cinco linhas da tabela
dados_clima.head()   



In [ ]:
print('\nMissing Values no dataframe:\n',dados_clima.isnull().sum(), sep="")

In [ ]:
#Tratar os Missing Values (Valores Ausentes)

dados_clima['Temp Mín (°C)'].fillna(method='ffill',inplace = True)   #ffill => preencher com a última temperatura antes do Missing Value
dados_clima['Temp Média (°C)'].fillna(method='ffill',inplace = True) #ffill => preencher com a última temperatura antes do Missing Value
dados_clima['Velocidade do Vento (km/h)'].fillna(method='ffill',inplace = True) #ffill => preencher com a última velocidade antes do Missing Value
dados_clima['Radiação solar (cal/cm²/h)'].fillna(method='ffill',inplace = True) #ffill => preencher com a última radiação antes do Missing Value
dados_clima['UR (%)'].fillna(method='ffill',inplace = True) #ffill => preencher com a última taxa UR antes do Missing Value
dados_clima['Chuva (mm)'].fillna(method='ffill',inplace = True) #ffill => preencher com a última quantidade antes do Missing Value
dados_clima['Pressão (mb)'].fillna(method='ffill',inplace = True) #ffill => preencher com a última pressão antes do Missing Value
dados_clima['Choveu?'].fillna(method='ffill',inplace = True) #ffill => preencher com a última pressão antes do Missing Value
dados_clima['Temp Máx (°C)'].fillna(method='ffill',inplace = True)   #ffill => preencher com a última temperatura antes do Missing Value
#dados_clima['Choveu?'] = dados_clima['Choveu?'].astype(int)


#verificar os missing values após a imputação acima
print('\nMissing Values no dataframe  após o tratamento:\n',dados_clima.isnull().sum(), sep="")

In [ ]:
# Usamos a correlação de Pearson e ordenamos os valores
columns_corr = dados_clima.corr(method='pearson')['Chuva (mm)'].sort_values()
print(columns_corr)

In [ ]:
# Contar os dias de chuva e estiagem
contagem_chuva = dados_clima['Choveu?'].value_counts()

# Resultados
dias_chuva = contagem_chuva.get(1, 0)  # Se não houver valor 1, retorna 0
dias_estiagem = contagem_chuva.get(0, 0)  # Se não houver valor 0, retorna 0

print(f"Total de dias que choveu: {dias_chuva}")
print(f"Total de dias de estiagem: {dias_estiagem}")

In [ ]:
dados_clima.describe()

## Time Period Selection for Analysis

In [ ]:
# 70% para treino / 30% para teste

data_inicio = 1980
data_fim = 2020
data_inicio_treinamento = 2009
data_fim_treinamento = 2019
data_inicio_teste = 1983
data_fim_teste = 2008




In [ ]:
dados_clima.drop(dados_clima.loc[dados_clima['Chuva (mm)']==0].index, inplace=True)

In [ ]:
dadosparateste = dados_clima.copy()

#Criando uma nova coluna de chuva para ficar no final do dataframe

qtdchuva = dadosparateste['Chuva (mm)']


dadosparateste['Chuva att (mm)'] = qtdchuva

#Visualização das primeiras cinco linhas da tabela
dadosparateste.head()  

In [ ]:
dadosparateste.drop(columns = ['Chuva (mm)'], axis = 1, inplace=True)


## Shared Utility Functions for All Models

In [ ]:
# Library para statmodels
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_percentage_error

#Definir funções que serão utilizadas por todos os modelos de predição

#Imprimir mensagens de erro da predição
def imprimir_scores_predicao(strModel, y_target, arPredict):
  #print(strModel, ' - R2: ', r2_score(y_target, arPredict))
  print(strModel, ' - MAE:  ', mean_absolute_error(y_target, arPredict))
  print(strModel, ' - MAPE: ', mean_absolute_percentage_error(y_target, arPredict))
  #print(strModel, ' - MSE: ', mean_squared_error(y_target, arPredict))
  print(strModel, ' - RMSE: ', np.sqrt(mean_squared_error(y_target, arPredict)))
  print(strModel, ' - MSE: ',mean_squared_error(y_target, arPredict))
  print(strModel, ' - R2: ',r2_score(y_target, arPredict))
  #mse = metrics.mean_squared_error(y, yhat)
  
  
def imprimir_graficos_predicao(strModel, y_target, arPredict):
  
  df_Test_Predicted = y_target.copy()
  df_Test_Predicted['Daily rain (mm)'] = arPredict

  #Plotagem dos dados de teste e dados previstos
  plt.figure(figsize=(16,8))
  plt.plot(y_target.loc['Dia'], y_target, color = 'blue', label = 'Chuva Real')
  plt.plot(y_target.loc['Dia'], arPredict, color = 'red', label = 'Chuva Prevista')
  font = {'family': 'serif',
        'color':  'black',
        'weight': 'normal',
        'size': 13.5,
        }
  plt.title('Comparação entre o real e o previsto pelo modelo', fontdict=font)
  plt.xlabel('Período')
  plt.ylabel('Quantidade de chuva')
  plt.legend()
  plt.grid(True)
  #plt.savefig('lstm2.pdf')
  plt.show() 

# Machine Learning Models - Regression

In [ ]:
# Criar os dataframes de Treinamento e Teste
df_train = dadosparateste[(dadosparateste['Ano'] >= data_inicio_treinamento) & (dadosparateste['Ano'] <= data_fim_treinamento)]
df_test = dadosparateste[(dadosparateste['Ano'] >= data_inicio_teste) & (dadosparateste['Ano'] <= data_fim_teste)]

# Separar variáveis independentes (X) e variável alvo (y)
X_train = df_train.iloc[:, 0:7]
y_train = df_train[['Chuva att (mm)']]

X_test = df_test.iloc[:, 0:7]
y_test = df_test[['Chuva att (mm)']]

# Exibir as primeiras linhas do conjunto de teste
X_train.head()


# XGBOOST 

In [ ]:
import xgboost as xgb

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 100, alpha = 1, n_estimators = 100000)

In [ ]:
xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)

In [ ]:
imprimir_scores_predicao("XGBoost", y_test, preds)


In [ ]:
from matplotlib import pyplot
print(xg_reg.feature_importances_)
# plot
pyplot.bar(range(len(xg_reg.feature_importances_)), xg_reg.feature_importances_)
pyplot.show()

In [ ]:
from xgboost import plot_importance
plot_importance(xg_reg)
pyplot.show()

In [ ]:
chuvaprevista = []
for z in range(len(preds)):
    #print(y_pred4[z])
    chuvaprevista.append(preds[z])
    
df_test['Previsão']= chuvaprevista
df_test.tail()    

In [ ]:

df_test = df_test[df_test['Ano']>= 1980]
df_test =  df_test[df_test['Ano'] <= 2020]

In [ ]:
plt.figure(figsize=(10,10))

true_value = df_test['Chuva att (mm)']
predicted_value = df_test['Previsão']
plt.scatter(true_value, predicted_value, c='crimson')
plt.yscale('log')
plt.xscale('log')

p1 = max(max(predicted_value), max(true_value))
p2 = min(min(predicted_value), min(true_value))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('True Values', fontsize=15)
plt.ylabel('Predictions', fontsize=15)
plt.axis('equal')
plt.show()